In [86]:
import requests
from bs4 import BeautifulSoup as bs
import re
from datetime import date
import os.path
import os
import pprint
import pandas as pd


def non_google_links(href):
    article = re.compile('http')
    google = re.compile('google')
    if href is None:
        return False
    elif article.search(href) and not google.search(href):
        return True
    else:
        return False


def google_top_results(n, ext):
    url = 'https://www.google.com' + ext
    page = requests.get(url)
    soup = bs(page.content, 'lxml')
    main = soup.find(id='main')
    links = main.find_all(href=non_google_links)
    news = []
    for link in links:
        clean_link = link.get('href')
        regex = re.compile(r'(http[s]?://.*?)&sa')
        m = regex.search(clean_link)
        news.append(m.group(1))
    news = pd.Series(news)
    news = pd.Series(news.unique())
    return news[:n]

def html_to_string(url):
    # connect to url and transrom to soup
    page = requests.get(url)
    soup = bs(page.content, 'lxml')
    # Get rid of links in page
    links = soup('a')
    for link in links:
        link.decompose()
    #extract the text
    text = soup.text
    return text

def clean_news(text, words4paragraph):
    #break text into elements based on blank lines
    regex = re.compile('^[\n\r]', re.MULTILINE)
    clean_text = pd.Series(regex.split(text))
#     cleaner = clean_text.replace('\t', '').replace('\n','').replace('\r', '')
    cleaner = clean_text.str.replace(' +', ' ')
    clean = cleaner[cleaner.str.count(' ')>=words4paragraph]
    return clean


def save_file(type_of_file, num, url, text):
    today = str(date.today())
    num += 1
    file_name = type_of_file+'_'+str(num)+'_'+today+'.txt'
    if os.path.isfile(file_name):
        os.remove(file_name)
    f = open(file_name, 'a')
    f.write(url+'\n')
    for k in text:
        f.write(k + '\n\n')
    f.close

In [87]:
f = open('google_news/test/google_news_1_2020-08-25.txt', 'r')
text = f.read()
news = clean_news(text, 20)
save_file('google_news/google_news', 1, 'www.fuckoff.com', news)
f.close()

test = news.str.replace(' +', ' ')
news[1]
# test[1]

' Sections xml:space="preserve"> Search xml:space="preserve"> Advertisement Advertisement Coronavirus in Illinois updates: Rate of people testing positive for COVID-19 remains high in Downstate counties despite new limits By Chicago Tribune | Aug 25, 2020 at 9:51 AM The rate of people testing positive for the coronavirus remains high in several Downstate counties despite new restrictions placed on public gatherings, according to public health officials. The “mitigation efforts” were imposed last Tuesday after the Metro East region near St. Louis reported a seven-day average positivity rate of above 8% for three consecutive days. As of Monday, the region’s seven-day positivity rate was 9.4%, down only slightly from the 9.6% reported on Sunday. Advertisement The Illinois Department of Public Health said increased restrictions were likely coming to the region this week, including the closure of indoor services at bars and restaurants. But officials say that action has been delayed after c

In [24]:
print(google_news.index)

RangeIndex(start=0, stop=3, step=1)
